In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
train_data=pd.read_csv("train.txt", delimiter=",", header=None,names=['Neighbours'],index_col=False)

In [21]:
train_data['ID']=train_data['Neighbours'].apply(lambda x: x.split('\t')[0])  #get the ID

In [22]:
train_data['Neighbours']=train_data['Neighbours'].apply(lambda x: x.split('\t')[1:])# set up the neighbours

In [23]:
train_data=train_data[["ID","Neighbours"]] # set "ID" as the index
train_data

,ID,Neighbours
0,540762,"[1912140, 1537559, 3091331, 2757277, 3237295, ..."
1,2129843,"[65840, 3414168, 4523797, 2851163, 4321895, 13..."
2,3361377,"[955840, 3342058, 1536902, 1850727, 1504632, 1..."
3,1199298,"[2300061, 2635670, 2803600, 744722, 881446, 28..."
4,1392121,"[3845572, 546016, 4361302, 678461, 4294597, 24..."
...,...,...
19995,585576,"[660302, 3279973, 2094235, 2355188, 1296935, 3..."
19996,505961,"[3875645, 2148630, 4288909, 4011139, 340232, 1..."
19997,125824,"[54421, 868022, 385000, 2050130, 3446665, 2040..."
19998,896087,"[431577, 1007572, 499457, 3642500, 3734728, 28..."
